# Trade Network Analysis
`09_generate_panel.ipynb`

---

METODO:
1. take log of series
2. Fix IMF series! include 2007
3. Drop unbalanced

#### Libraries

In [1]:
import pandas as pd                                         # pandas dataframes used for convenience
from pandas.stats.plm import PanelOLS
import numpy as np
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

Load IMF data

In [2]:
decomp = pd.read_csv('imports_decomp.csv')
decomp = decomp.rename(columns={'Unnamed: 0': 'date', 'j': 'iso2'})
decomp['date'] = decomp['date'].apply(str)
decomp = decomp.set_index(['date', 'iso2'])
df = pd.read_csv('imf_data.csv', index_col=[0,1,2,3], skipinitialspace=True).reset_index()[2:]
df = df.rename(columns={'level_0': 'iso2', 'Unnamed: 1': 'full_name', 'Unnamed: 2':'date', 'mp': 'mp_base', 
                        'mp.1': 'mp',  'xp': 'xp_base', 'xp.1': 'xp'})
df = df.set_index(['date', 'iso2'])
panel = df.join(decomp)
for v in ['q', 'x', 'xp', 'mp']:
    panel[v] = panel[v].apply(float) # replace country names

#### Deflate USD values with price indices

In [3]:
deflators = {'x': 'xp', 'cs': 'mp', 'ns': 'mp', 'cn': 'mp', 'nn': 'mp'}
for k, v in deflators.iteritems():
    panel[k+'_def'] = np.log(panel[k] * (panel[v] / 100))
#panel['q'] = np.log(panel['q'])

In [4]:
panel['x_ch'] = panel['x_def'].groupby(df.index.get_level_values(1)).diff(periods=-1)
panel = panel.dropna()

In [11]:
import statsmodels.api as sm 
import statsmodels.formula.api as smf

In [7]:
panel.to_csv('panel.csv')

In [5]:
#panel.dropna()[['q', 'x_def', 'cs_def', 'ns_def', 'cn_def', 'x_ch']]
#from pandas.stats.plm import PanelOLS

In [8]:
reg  = PanelOLS(y=panel['x_ch'],x=panel[['x_def', 'q', 'cs_def', 'ns_def', 'cn_def']],time_effects=True)
reg

C:\Users\BDew\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2825: FutureWarning: The pandas.stats.plm module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://statsmodels.sourceforge.net/stable/mixed_linear.html
  if self.run_code(code, result):



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x_def> + <q> + <cs_def> + <ns_def> + <cn_def>

Number of Observations:         602
Number of Degrees of Freedom:   12

R-squared:         0.0196
Adj R-squared:     0.0013

Rmse:              0.1973

F-stat (5, 590):     1.0728, p-value:     0.3743

Degrees of Freedom: model 11, resid 590

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
         x_def     0.0218     0.0104       2.10     0.0363     0.0014     0.0421
             q     0.0009     0.0012       0.74     0.4616    -0.0015     0.0032
        cs_def     0.0156     0.0084       1.86     0.0640    -0.0009     0.0322
        ns_def    -0.0211     0.0108      -1.96     0.0508    -0.0422     0.0000
        cn_def    -0.0165     0.0121      -1.3